#  Praktikum : Decision Trees und Ensembles - California Housing Datasets. 

Um sicherzustellen, dass alle notwendigen Python-Dependencies zur Verfügung stehen, arbeiten wir in diesem Praktikum in einem Docker Container. 

**Vorgehen: **
- Laden Sie das Jupyter Notebook von Moodle in einen ML1-Praktikums-Ordner in Ihrem User-Ordner. 
- Geben Sie im Terminal ein: "docker_start_ml1_gpu". Dadurch wird ein Skript aufgerufen, das einen Docker-Container für Sie baut. Das Terminal bleibt offen, wechselt nun allerdings in einen Terminal des Docker-Containers.
- Öffnen Sie den angegebenen localhost:8888-Link mit Token im Browser. Ein Jupyter Notebook sollte sich öffnen. Unter dem Ordner "host" finden Sie Ihr Homedirectory und auch das Jupyter-Notebook für das Praktikum. 
- Zum Ende des Praktikums speichern Sie Ihre Fortschritte, und beenden dann den Docker Container mit der Eingabe von "exit" im Terminal. 

Wir laden wieder die vorbereiteten Trainins- und Test-Daten housing_train.csv und housing_test.csv als DataFrames housing_train und housing_test, sowie die Labels train_labels.csv und test_labels.csv als train_labels und test_labels.  

In [2]:
import os
import pandas as pd
import numpy as np

HOUSING_PATH = os.path.join("datasets", "housing")

path_train_labels = os.path.join(HOUSING_PATH, "train_labels.csv")
path_test_labels = os.path.join(HOUSING_PATH, "test_labels.csv")
path_test = os.path.join(HOUSING_PATH, "housing_test.csv")
path_train = os.path.join(HOUSING_PATH, "housing_train.csv")
train_labels = pd.read_csv(path_train_labels)
test_labels = pd.read_csv(path_test_labels)
housing_train = pd.read_csv(path_train)
housing_test = pd.read_csv(path_test)


### Allgemeiner Hinweis zum Trainieren von Modellen: Der Hyperparameter n_jobs

Wenn Sie eine Instanz eines Modells aufrufen (z.B. wenn Sie log_reg = LogisticRegression() setzen, oder bei Grid Search...), gibt es den Hyperparameter n_jobs. Dieser gibt an, wie viele Rechenschritte Sie parallel ausführen lassen wollen, für den Fall, dass Sie entsprechende Rechenpower besitzen. Wenn Sie die maximal mögliche Anzahl von Rechenschritten parallelisierten wollen, setzen Sie n_jobs=-1. 

###  Decision Trees

Beantworten Sie zunächst in der nächsten Markdown-Zelle folgende Fragen: 
1. Beschreiben Sie die Funktionsweise eines Decision Trees!
2. Was beschreibt der Gini Index? Was ist der Index für ein reines bzw. gleichverteiltes Datenset bei C Klassen?
3. Was sind Vor- und Nachteile von Decision Trees? Nennen Sie jeweils einen!

1. #TODO: 
   
2. #TODO: 
   
3. #TODO: 

Nun verwenden wir einen Decision Tree statt eines Linearen Modells für obigen ML task: 
(https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

Frage: Was passiert, wenn Sie den Decision Tree völlig "unkontrolliert" wachsen lassen? 


#TODO:

Trainieren sie ein regularisiertes Decision Tree Modell tree_reg.
Es gibt mehrere Möglichkeiten, das Wachstum des Baumes einzuschränken, um Overfitting zu vermeiden: 
- max_depth: stoppt das Splitten, wenn diese Tiefe erreicht wird
- max_leaves: stoppt das Splitten, wenn diese Anzahl an Nodes erreicht wird
- min_samples_leaf: stoppt das Splitten, wenn an einem Leaf diese minimale Anzahl von Trainings-Daten erreicht wird 
- min_samples_split: stoppt das Splitten von internal nodes, wenn hier diese minimale Anzahl von Training Samples erreicht wird.

Visualisieren Sie den Decision Tree tree_reg mit export_graphviz. (https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html). Falls Sie nicht in dem Docker-Container arbeiten, müssen Sie evtl erst graphviz installieren. Achtung: hier haben wir einen Regression Tree, das heißt wir brauchen hier anders als im Notebook der Vorlesung keine class_names zu spezifizieren! Die resultierende Datei ist ein dot file, die Sie bitte unter "california_housing_tree.dot" unter dem HOUSING_PATH abspeichern. Diese Datei muss man im nächsten Schritt in eine Bilddatei konvertieren. Dazu geben Sie 

dot -Tpng california_housing_tree.dot -o california_housing_tree.png

in den Terminal ein. Dies generiert eine california_housing_tree.png. 



In [ ]:
from sklearn.tree import export_graphviz

# def image_path(fig_id):
#     return os.path.join(IMAGES_PATH, fig_id)

export_graphviz(
        tree_reg,
        out_file=os.path.join(HOUSING_PATH, "california_housing_tree.dot"),
        feature_names=housing_train.columns,
        rounded=True,
        filled=True
    )

Lassen Sie diese Datei california_housing_tree.png im folgenden Markdown-Fenster anzeigen mit
"![Decision_Tree](hier den Pfad von california_housing_tree.png eingeben)"

Unten ist eine Funktion `plot_learning_curve(estimator, X, y, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5))`, die Learning Plots für einen `estimator` und Trainingsdaten `X,y` zeichnet, mithilfe von Sklearn's [Learning_Curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html) mit cv=3 und den obigen Argumenten (3-fold cross validation; dadurch wird mit den Train-Daten 3-fold Cross Validation betrieben; ausgegeben werden als train_scores und test_scores (für DecisionTreeRegressors wäre das der $R^2$-Score) für jede Trainingsset-Größe drei Werte. Nehmen Sie von diesen den Mittelwert für den Plot).

Plotten Sie damit die Learning Curve für drei DecisionTrees (mit random_state=42 für Reproduzierbarkeit): 
einen mit `max_depth = 3`, einen mit `max_depth=5`, und einen ohne jegliche Regularisierung. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# from sklearn.svm import SVC
# from get_csv_data import HandleData
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, X, y, ax=None, ylim=(0.5, 1.01),  n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):

    train_sizes, train_scores, test_scores = \
        learning_curve(estimator, X, y, cv=3, n_jobs=n_jobs, train_sizes=train_sizes)
              
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

    # Plot learning curve
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    ax.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    ax.set_ylim(ymin = 0, ymax= 1)
    ax.legend(loc="best")

    return plt

# define three models and plot the learning curves.



Erklären Sie die Graphen. Welcher Baum ist der beste?

#TODO: 

Trainieren Sie das beste Modell von eben mit allen Trainingsdaten, und vergleichen Sie den sich ergebenden R^2-Score auf den Test-Daten mit dem des unregularisierten Linearen Regressions-Modells vom letzten Mal. Was ist besser? 

#TODO

###  Random Forest 

Beantworten Sie zunächst in der nächsten Markdown Zelle folgende Fragen: 
1. Wie funktioniert ein Random Forest?
2. Was sind Bias und Variance eines Random Forests im Vergleich zum Basis-Modell? Was ist der Vorteil?

1. #TODO:  
   
2. #TODO: 

Trainieren Sie nun mit den Daten einen Random Forest Regressor forest_reg (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), also ein Decision Tree Ensemble aus 100 Decision Trees (n_estimators=100) mit derselben Tiefe wie das beste DecisionTreeRegression Modell zuvor. Berechnen Sie wieder den R^2-Score auf dem test set. Vergessen Sie nicht n_jobs=-1 zu setzen.

Hinweis: im Fall einer DataConversionWarning: train_labels ist aktuell ein Spaltenvektor, also arrays der Dimension (Anzahl Instances, 1), RandomForestRegressor hätte jedoch gerne einen Zeilenvektor als input. Sie erreichen dies mit Hilfe von numpy.ravel(). (https://numpy.org/doc/stable/reference/generated/numpy.ravel.html)


In [ ]:
# flatten train_labels und test_labels first
train_labels_flat = np.ravel(train_labels) 
test_labels_flat = np.ravel(test_labels) 

# Trainieren des RandomForestRegressors


Für den Fall, dass es mehr als einen Hyperparameter zu optimieren gibt, wird "Ausprobieren" zeitaufwändig. 
Verwenden Sie GridSearchCV mit 5-fold Cross-Validation, um für n_estimators = 100 die besten Werte für folgenden Grid an Hyperparametern zu finden: max_depth zwischen 4 und 7, max_features zwischen 4 und 10 in Zweierschritten, und min_samples_leaf 5, 10 oder 15. (n_jobs=-1)
Lassen Sie sich die besten Parameter ausgeben, sowie die informationen über den besten Estimator mit .best_estimator_. Berechnen Sie wieder den R^2-Score des best_estimator_ auf den Testdaten. Welches Modell ist jetzt das beste?

In [ ]:
param_grid = [
    {'max_depth': [4,5,6,7], 'max_features': [4, 6, 8, 10], 'min_samples_leaf': [5,10,15]},
  ]




Welche der Features in sind die relevanten? Plotten Sie die Feature Importance des best_estimator_. 

#  Praktikum: Boosting


Trainieren Sie eine GradientBoostingRegressor gbrt mit den obigen besten Baum-Parametern und berechnen Sie den resultierenden R^2-Score auf den Test Daten. Hinweis: GradientBoostingRegressor will wieder einen Zeilenvektor als input. Ist das Modell besser oder schlechter als das Random Forest Modell final_forest?

Doku: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html# sklearn.ensemble.GradientBoostingRegressor

## Rechenaufgabe

NASA wants to be able to discriminate between Martians (M) and Humans (H) based on the following characteristics: Green $\in \{N,Y\}$, Legs $\in \{2,3\}$, Height $\in \{S,T\}$, Smelly $\in \{N,Y\}$. The available training data $D$ is as follows: 


|Species |Green | Legs |Height | Smelly|
|---|---|---|---|---|
|1| M|N|3|S|Y|
|2|M|Y|2|T|N|
|3|M|Y|3|T|N|
|4|M|N|2|S|Y|
|5|M|Y|3|T|N|
|6|H|N|2|T|Y|
|7|H|N|2|S|N|
|8|H|N|2|T|N|
|9|H|Y|2|S|N|
|10|H|N|2|T|Y|

- Compute the Gini impurity of the dataset. 
- Consider the features "legs" and "smelly", and compute the Gini impurity in case the entire dataset is split up according to each of those two features. Which of those will rather be used for the first split?


#TODO

## Fragen

True or false? 
- If a decision tree performs badly on the test data, scaling the input can help. 
- If a decision trees performs badly on validation data, it can help to limit max_depth. 

#TODO